In [ ]:
import dotenv
import requests

dotenv.load_dotenv()

In [3]:
import requests
import os

# api_key = os.getenv("BRAVE_API_KEY")
api_key = 'BSAKiixseKVtNfNdfy0NKTIatrmReBV'
query = "Ibrahim Cisse Google Scholar"
search_url = "https://api.search.brave.com/res/v1/web/search"
headers = {
    "Accept": "application/json",
    "X-Subscription-Token": api_key
}
params = {"q": query, "count": 10}


response = requests.get(search_url, headers=headers, params=params)
print(response.status_code)
if response.status_code == 200:
    results = response.json()

results

200


{'query': {'original': 'Ibrahim Cisse Google Scholar',
  'show_strict_warning': False,
  'is_navigational': False,
  'is_news_breaking': False,
  'spellcheck_off': True,
  'country': 'us',
  'bad_results': False,
  'should_fallback': False,
  'postal_code': '',
  'city': '',
  'header_country': '',
  'more_results_available': True,
  'state': ''},
 'mixed': {'type': 'mixed',
  'main': [{'type': 'web', 'index': 0, 'all': False},
   {'type': 'web', 'index': 1, 'all': False},
   {'type': 'web', 'index': 2, 'all': False},
   {'type': 'web', 'index': 3, 'all': False},
   {'type': 'web', 'index': 4, 'all': False},
   {'type': 'web', 'index': 5, 'all': False},
   {'type': 'web', 'index': 6, 'all': False},
   {'type': 'web', 'index': 7, 'all': False},
   {'type': 'web', 'index': 8, 'all': False},
   {'type': 'web', 'index': 9, 'all': False},
   {'type': 'web', 'index': 10, 'all': False},
   {'type': 'web', 'index': 11, 'all': False},
   {'type': 'web', 'index': 12, 'all': False},
   {'type': '

In [6]:
results['web']['results']

[{'title': '\u202aIbrahim Cissé\u202c - \u202aGoogle Scholar\u202c',
  'url': 'https://scholar.google.fr/citations?user=SsuqMfwAAAAJ&hl=en',
  'is_source_local': False,
  'is_source_both': False,
  'description': '\u202aMax Planck Institute, Freiburg\u202c - \u202a\u202aCited by 8,152\u202c\u202c - \u202aPhysics\u202c - \u202aBiophysics\u202c',
  'profile': {'name': 'Google',
   'url': 'https://scholar.google.fr/citations?user=SsuqMfwAAAAJ&hl=en',
   'long_name': 'scholar.google.fr',
   'img': 'https://imgs.search.brave.com/lAgSFkWd3KU9t2s5R0IHMRLSRPbceyLXsaMmS--Mix4/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvZjA5NWY5ZTYx/ZDdmZGIyZDIzMDEy/Y2M0YzgwNzc3ODNi/Y2U2MTA4MTk5ODIx/YTY3NDgzYzVmZTk2/ZmJhOTNiNi9zY2hv/bGFyLmdvb2dsZS5m/ci8'},
  'language': 'en',
  'family_friendly': True,
  'type': 'search_result',
  'subtype': 'generic',
  'is_live': False,
  'meta_url': {'scheme': 'https',
   'netloc': 'scholar.google.fr',
   'hostname': 'scholar.google.fr',
   'f

In [7]:
filtered_results = [
    {"title": item["title"], "url": item["url"], "description": item.get("description", "No description available")}
    for item in results["web"]["results"]
]

# Print the extracted data
for res in filtered_results:
    print(f"Title: {res['title']}")
    print(f"URL: {res['url']}")
    print(f"Description: {res['description']}\n")

Title: ‪Ibrahim Cissé‬ - ‪Google Scholar‬
URL: https://scholar.google.fr/citations?user=SsuqMfwAAAAJ&hl=en
Description: ‪Max Planck Institute, Freiburg‬ - ‪‪Cited by 8,152‬‬ - ‪Physics‬ - ‪Biophysics‬

Title: Ibrahim Cissé (academic) - Wikipedia
URL: https://en.wikipedia.org/wiki/Ibrahim_Ciss%C3%A9_(academic)
Description: Ibrahim I. Cissé is a <strong>Nigerien-American biophysicist</strong>. He is currently director of the Max Planck Institute of Immunobiology and Epigenetics. Previously, Cissé was at the California Institute of Technology as Professor of Physics and the Massachusetts Institute of Technology as Professor of ...

Title: The Cissé Laboratory @ MIT
URL: http://www.icisse.org/
Description: <strong>Endowed Physics Professorship</strong>: Ibrahim has been named to hold the Class of 1922 Career Development Chair at MIT.

Title: Ibby (@IbrahimCisse_) / Twitter
URL: https://twitter.com/ibrahimcisse_
Description: The latest tweets from Ibby (@IbrahimCisse_)

Title: Fueling prote

In [9]:
import re

def remove_html_tags(text):
    return re.sub(r"</?strong>", "", text)


formatted_results = "\n".join(
    f"Title: {remove_html_tags(res['title'])}\n"
    f"URL: {res['url']}\n"
    f"Description: {remove_html_tags(res['description'])}\n"
    for res in filtered_results
)

print(formatted_results)

Title: ‪Ibrahim Cissé‬ - ‪Google Scholar‬
URL: https://scholar.google.fr/citations?user=SsuqMfwAAAAJ&hl=en
Description: ‪Max Planck Institute, Freiburg‬ - ‪‪Cited by 8,152‬‬ - ‪Physics‬ - ‪Biophysics‬

Title: Ibrahim Cissé (academic) - Wikipedia
URL: https://en.wikipedia.org/wiki/Ibrahim_Ciss%C3%A9_(academic)
Description: Ibrahim I. Cissé is a Nigerien-American biophysicist. He is currently director of the Max Planck Institute of Immunobiology and Epigenetics. Previously, Cissé was at the California Institute of Technology as Professor of Physics and the Massachusetts Institute of Technology as Professor of ...

Title: The Cissé Laboratory @ MIT
URL: http://www.icisse.org/
Description: Endowed Physics Professorship: Ibrahim has been named to hold the Class of 1922 Career Development Chair at MIT.

Title: Ibby (@IbrahimCisse_) / Twitter
URL: https://twitter.com/ibrahimcisse_
Description: The latest tweets from Ibby (@IbrahimCisse_)

Title: Fueling protein–DNA interactions inside porous 